# Using NP4VTT: ANN-based VTT model

In this notebook, we show how to use an Artificial Neural Network (ANN) based method to estimate the distribution of the Value of Travel Time (VTT) from the Norway data.

## Step 1: Load modules and data, and create arrays

We first import the NP4VTT modules for creating the arrays, the ANN, and Pandas to load the dataset:

In [1]:
import sys
sys.path.append('../src')

In [2]:
import pandas as pd

from py_np4vtt.data_format import StudyVar
from py_np4vtt.model_ann import ModelANN, ConfigANN
from py_np4vtt.data_import import make_modelarrays, compute_descriptives


Now we read the CSV file:

In [3]:
df = pd.read_table('../data/Norway09_data_v5.txt')
df.head()

,RespID,Mode,TravTime,BaseCost,Gender,AgeClass,IncClass,TravTimeClass,Purpose,CardID,...,TimeL,TimeR,Chosen,Quadrant2,Purpose2,Mode2,Income2,ExclGroup,Exclude_CDF,CS
0,88,1,25,27,1,3,5,1,1,1,...,32,25,1,4,1,1,3,4,1,1
1,88,1,25,27,1,3,5,1,1,2,...,25,28,2,2,1,1,3,4,1,2
2,88,1,25,27,1,3,5,1,1,3,...,29,25,1,4,1,1,3,4,1,3
3,88,1,25,27,1,3,5,1,1,4,...,32,25,1,2,1,1,3,4,1,4
4,88,1,25,27,1,3,5,1,1,5,...,29,32,2,2,1,1,3,4,1,5


The dataset contains 22 variables. Each row is a binary choice task. We will use:

* `RespID`: ID of each respondent.
* `Chosen`: Chosen alternative.
* `CostL`: Travel cost of alternatives 1 [NOK]
* `CostR`: Travel cost of alternatives 2 [NOK]
* `TimeL`: Travel time of alternatives 1 [minutes]
* `TimeR`: Travel time of alternatives 2 [minutes]

NP4VTT detects automatically the _slow and cheap_ and _fast and expensive_ alternative, for each choice situation. If NP4VTT finds violations to those two options (e.g., a fast-cheap) alternative, it will raise an error message.

Change currency of travel time to euros and change unit of travel time to hours

In [4]:
# Convert to euros
NOK2euro_exchange_rate = 9
df[['CostL','CostR']] = df[['CostL','CostR']] .div(NOK2euro_exchange_rate)

# convert to hours
df[['TimeL','TimeR']] = df[['TimeL','TimeR']] .div(60)

In [5]:
columnarrays = {
    StudyVar.Id: 'RespID',
    StudyVar.ChosenAlt: 'Chosen',
    StudyVar.Cost1: 'CostL',
    StudyVar.Cost2: 'CostR',
    StudyVar.Time1: 'TimeL',
    StudyVar.Time2: 'TimeR',
}

And we create the required arrays:

In [6]:
model_arrays = make_modelarrays(df, columnarrays)

The function `make_modelarrays` creates six elements used by NP4VTT to estimate/train a model: RIGHT

* `BVTT`: Contains the Boundary VTT per choice situation, computed from costs and time.
* `Choice`: A matrix of dummy variables that are equal to one if the respondent choose the fast-expensive alternative on each choice situation.
* `Accepts`: Number of times a respondent chose the fast-expensive alternative.
* `ID`: Unique identifier of each respondent.
* `NP`: Number of respondents in the dataset.
* `T`: Number of choice situations per respondent.

## Step 2: Compute descriptives

The function `compute_descriptives` provides a small overview of the dataset characteristics:

In [7]:
descriptives = compute_descriptives(model_arrays)
print(descriptives)

No. individuals: 5832
Sets per indiv.: 9

Number of non-traders:
Fast-exp. alt.: 144
Slow-cheap alt.: 808

BVTT statistics:
Mean chosen BVTT: 3.5800867802855794
Minimum of BVTT: 0.6666666666666644
Maximum of BVTT: 113.56321839080461


## Step 3: Configure an ANN

The ANN requires the following parameters from the user:

* `hiddenLayerNodes`: A list of length equal to the number of hidden layers. Each element is the number of hidden nodes of each hidden layer.
* `trainingRepeats`: Number of training repeats
* `shufflesPerRepeat`: Number of shuffles of the data (for randomisation of choice situation and data expansion)

Additionally, the user can configure the following options:
* `seed`: Random seed for shuffling and simulation.

The function `ConfigANN` takes the configuration parameters of the ANN and creates an object that is used by the optimisation routine:

In [8]:
config = ConfigANN(hiddenLayerNodes=[10, 10], trainingRepeats= 5, shufflesPerRepeat= 50, seed= None)

Now, we create the ANN object that contains the configuration parameters and the data arrays. Then, we initialise the arguments:

In [9]:
ann = ModelANN(config, model_arrays)
initialArgs = ann.setupInitialArgs()

## Step 4: Train an ANN

Once the ANN is initialised, the `run` method starts the optimisation process:

In [10]:
ll, r2, clf, vtt = ann.run(initialArgs)

Rep 1: CE (train): 0.3695 / CE (test): 0.3678 / LL: -107664.06 / Rho-sq: 0.47
Rep 2: 

AttributeError: 'numpy.ndarray' object has no attribute 'append'

The training process returns the following information:

* `ll:` A list with the log-likelihood of each repetition
* `r2:` A list with the Rho-squared of each repetition
* `clf:` A list with the `sklearn` trained model objects.
* `vtt:` A list with the estimated VTT per repetition

The following lines present the estimated results:

In [ ]:
import numpy as np

# Create dataframe
results = pd.DataFrame(np.c_[ll,r2],columns=['Log-likelihood','Rho-squared'],index=['Rep. ' + str(i+1) for i in range(len(ll))])

print('Training results:')
print(results)

## Step 5: Visualising the VTT distribution

We create the empirical Cumulative Density Function plot of the VTT distribution. For easy interpretation, we use create a histogram from the CDF. 
We use matplotlib for this:

In [ ]:
vtt_mean = vtt.mean(axis=0)

import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

fig, (ax1,ax2) = plt.subplots(1,2,sharex=True,figsize=(6,3), dpi=100)

# Histogram
ax1.hist(vtt_mean,bins=100,range=(0,20))

# ECDF
ecdf = ECDF(vtt_mean)
ax2.plot(ecdf.x,ecdf.y)

plt.tight_layout()